### Здесь будет сохранён код парсинга данных с сайта auto.ru

In [2]:
import requests
from bs4 import BeautifulSoup 
import pandas as pd
import re
import time
import json

Загрузим имеющийся датасет, который был собран с сайта auto.ru 09.09.2020, чтобы понять какие данные нас интересуют

In [2]:
data = pd.read_csv('test.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34686 entries, 0 to 34685
Data columns (total 32 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   bodyType              34686 non-null  object
 1   brand                 34686 non-null  object
 2   car_url               34686 non-null  object
 3   color                 34686 non-null  object
 4   complectation_dict    6418 non-null   object
 5   description           34686 non-null  object
 6   engineDisplacement    34686 non-null  object
 7   enginePower           34686 non-null  object
 8   equipment_dict        24690 non-null  object
 9   fuelType              34686 non-null  object
 10  image                 34686 non-null  object
 11  mileage               34686 non-null  int64 
 12  modelDate             34686 non-null  int64 
 13  model_info            34686 non-null  object
 14  model_name            34686 non-null  object
 15  name                  34686 non-null

Нам нужно будет спарсить тот же модельный ряд

In [3]:
brands = [brand.lower() for brand in data.brand.unique()]
brands

['skoda',
 'audi',
 'honda',
 'volvo',
 'bmw',
 'nissan',
 'infiniti',
 'mercedes',
 'toyota',
 'lexus',
 'volkswagen',
 'mitsubishi']

Сохраним все сыслки на авто из этого списка

In [4]:
def collect_car_urls(brand):
    
    '''Сохраняет все доступные url объявлений для моделей из списка, который подаётся на вход'''
    
    main_url = f'https://auto.ru/cars/{brand}/all/'   
    main_response = requests.get(main_url)   
    main_soap = BeautifulSoup(main_response.content.decode('utf-8'), 'html.parser')    
    _ = main_soap.find('span', class_='ButtonWithLoader__content').text.replace(u'\xa0', '')
    urls_total = int(re.findall(r'\d+', _)[0])
    ads_per_page = len(main_soap.find_all('a', class_='Link ListingItemTitle__link'))
    pages_num = urls_total // ads_per_page
    
    all_urls = []
    
    for page_num in range(1, pages_num):
        if page_num % 20 == 0:
            print(f"Extracting page {page_num} from {pages_num}...")
        page_url = f'{main_url}?page={page_num}'   
        page_response = requests.get(page_url)
        time.sleep(0.1)
        page_soap = BeautifulSoup(page_response.content.decode('utf-8'), 'html.parser')   
        
        all_urls.extend([a.get('href') for a in page_soap.find_all('a', class_='Link ListingItemTitle__link')])

    return all_urls

In [5]:
urls = []
for brand in brands:
    all_urls = collect_car_urls(brand)
    urls.extend(all_urls)

Extracting page 20 from 239...
Extracting page 40 from 239...
Extracting page 60 from 239...
Extracting page 80 from 239...
Extracting page 100 from 239...
Extracting page 120 from 239...
Extracting page 140 from 239...
Extracting page 160 from 239...
Extracting page 180 from 239...
Extracting page 200 from 239...
Extracting page 220 from 239...
Extracting page 20 from 277...
Extracting page 40 from 277...
Extracting page 60 from 277...
Extracting page 80 from 277...
Extracting page 100 from 277...
Extracting page 120 from 277...
Extracting page 140 from 277...
Extracting page 160 from 277...
Extracting page 180 from 277...
Extracting page 200 from 277...
Extracting page 220 from 277...
Extracting page 240 from 277...
Extracting page 260 from 277...
Extracting page 20 from 133...
Extracting page 40 from 133...
Extracting page 60 from 133...
Extracting page 80 from 133...
Extracting page 100 from 133...
Extracting page 120 from 133...
Extracting page 20 from 82...
Extracting page 40 fro

In [6]:
url_df = pd.DataFrame({'car_url': urls})
url_df.to_csv('url_df.csv', index=False)
url_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132258 entries, 0 to 132257
Data columns (total 1 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   car_url  132258 non-null  object
dtypes: object(1)
memory usage: 1.0+ MB


Теперь нужно из всех url извлечь все параметры авто, которые есть в тестовой выборке


In [7]:
columns = data.columns.to_list()
columns

['bodyType',
 'brand',
 'car_url',
 'color',
 'complectation_dict',
 'description',
 'engineDisplacement',
 'enginePower',
 'equipment_dict',
 'fuelType',
 'image',
 'mileage',
 'modelDate',
 'model_info',
 'model_name',
 'name',
 'numberOfDoors',
 'parsing_unixtime',
 'priceCurrency',
 'productionDate',
 'sell_id',
 'super_gen',
 'vehicleConfiguration',
 'vehicleTransmission',
 'vendor',
 'Владельцы',
 'Владение',
 'ПТС',
 'Привод',
 'Руль',
 'Состояние',
 'Таможня']

In [8]:
def extract_url_data(url, columns):
    
    '''По указанному URL возвращает список из указанных параметров '''

    response = requests.get(url)
    page = BeautifulSoup(response.content.decode('utf-8'), 'html.parser')
    
    try:
        catalog_url = page.find(
            'a', class_='Link SpoilerLink CardCatalogLink SpoilerLink_type_default').get('href')
        response_catalog = requests.get(catalog_url)
        page_catalog = BeautifulSoup(response_catalog.content.decode('utf-8'), 'html.parser')
    except:
        pass
    try:
        json_data_catalog = json.loads(
            page_catalog.find('script', type="application/json", id='initial-state').string)
    except:
        pass
    try:
        json_data_equip = json.loads(
            page.find('script', type="application/json", id='initial-state').string)
    except:
        pass
    try:
        data = json.loads(
            page.find('script', type="application/ld+json").string)
        data = flatten(data)
    except:
        pass
    
    try:
        data['complectation_dict'] = [
        k for k, v in json_data_catalog['state']['compare']['selected'][0]['options'].items() if v == 1]
    except:
        pass
    try:
        data['equipment_dict'] = json_data_equip['card']['vehicle_info']['equipment']
    except:
        pass
    try:
        mileage = page.find(
            'li', class_='CardInfoRow CardInfoRow_kmAge').find_all('span')[-1].text.replace(u'\xa0', u'')
        data['mileage'] = int(re.findall(r'\d+', mileage)[0])
    except:
        pass
    try:
        data['model_name'] = page.find_all(
        'div', class_='InfoPopup InfoPopup_theme_plain InfoPopup_withChildren BreadcrumbsPopup')[1].text
    except:
        pass
    try:
        data['parsing_unixtime'] = int(time.time())
    except:
        pass
    try:
        data['sell_id'] = int(re.findall(
            r'\d+', page.find('div', class_='CardHead__infoItem CardHead__id').text)[0])
    except:
        pass
    try:
        data['super_gen'] = json.loads(
            page.find('div', id="sale-data-attributes").get('data-bem'))
    except:
        pass
    try:
        data['Владельцы'] = page.find(
            'li', class_='CardInfoRow CardInfoRow_ownersCount').find_all('span')[-1].text.replace(u'\xa0', u' ')
    except:
        pass
    try:
        data['Владение'] = page.find(
            'li', class_='CardInfoRow CardInfoRow_owningTime').find_all('span')[-1].text
    except:
        pass
    try:
        data['ПТС'] = page.find(
            'li', class_='CardInfoRow CardInfoRow_pts').find_all('span')[-1].text
    except:
        pass
    try:
        data['Привод'] = page.find(
            'li', class_='CardInfoRow CardInfoRow_drive').find_all('span')[-1].text
    except:
        pass
    try:
        data['Руль'] = page.find('li', class_='CardInfoRow CardInfoRow_wheel').find_all('span')[-1].text
    except:
        pass
    try:
        data['Состояние'] = page.find(
            'li', class_='CardInfoRow CardInfoRow_state').find_all('span')[-1].text
    except:
        pass
    try:
        data['Таможня'] = page.find(
            'li', class_='CardInfoRow CardInfoRow_customs').find_all('span')[-1].text
    except:
        pass
    try:
        data['description'] = re.sub('\W+', ' ', data['description'])
    except:
        pass
   
    output = []
    try:
        for col in columns:
            output.append(data.get(col, None))    
    except:
        pass
    if not output:
        output = [None] * len(columns)
    return output

In [11]:
parsed_list = []
for n, url in enumerate(url_df.car_url.values.tolist()[:]):
    if n % 50 == 0:
        print(f"The # of current processing URL is {n}, url is {url}.")
    if n % 1000 == 0:
        print(f"Create a dataframe from the list and concetenate it with the already collected data.")
        df  = pd.DataFrame(data=parsed_list, columns=columns + ['price'])
        parsed_df_1 = pd.concat([parsed_df_1, df], ignore_index=True)
        parsed_df_1.to_csv('parsed_data_1.csv', index=False)
        parsed_list = []
    parsed_list.append(extract_url_data(url, columns + ['price']))

The # of current processing URL is 0, url is https://auto.ru/cars/new/group/audi/a6/21791690/21793195/1105307125-6a5fce11/.
Create a dataframe from the list and concetenate it with the already collected data.
The # of current processing URL is 50, url is https://auto.ru/cars/used/sale/audi/q5/1105245708-08c2b8d6/.
The # of current processing URL is 100, url is https://auto.ru/cars/new/group/audi/a6/21791791/21793232/1105307145-8e9854a8/.
The # of current processing URL is 150, url is https://auto.ru/cars/new/group/audi/q5/22803801/22804040/1105296311-7faf66ee/.
The # of current processing URL is 200, url is https://auto.ru/cars/used/sale/audi/q7/1105641655-581f0f54/.
The # of current processing URL is 250, url is https://auto.ru/cars/new/group/audi/a6/21791791/21793232/1105307139-d0cb2bda/.
The # of current processing URL is 300, url is https://auto.ru/cars/used/sale/audi/q7/1105844018-10c112eb/.
The # of current processing URL is 350, url is https://auto.ru/cars/new/group/audi/a6/2179

The # of current processing URL is 3450, url is https://auto.ru/cars/used/sale/audi/q5/1105865823-9633a83f/.
The # of current processing URL is 3500, url is https://auto.ru/cars/used/sale/audi/q8/1103973093-8bac720a/.
The # of current processing URL is 3550, url is https://auto.ru/cars/used/sale/audi/q7/1105780672-130f8736/.
The # of current processing URL is 3600, url is https://auto.ru/cars/new/group/audi/a6/21791690/21793195/1104074204-0405f912/.
The # of current processing URL is 3650, url is https://auto.ru/cars/used/sale/audi/q8/1103973093-8bac720a/.
The # of current processing URL is 3700, url is https://auto.ru/cars/used/sale/audi/a6/1105933064-b771d330/.
The # of current processing URL is 3750, url is https://auto.ru/cars/used/sale/audi/q7/1105667446-8a81698e/.
The # of current processing URL is 3800, url is https://auto.ru/cars/used/sale/audi/a8/1105614015-55a1ce7e/.
The # of current processing URL is 3850, url is https://auto.ru/cars/used/sale/audi/a4/1105446897-41410987/.
T

The # of current processing URL is 7000, url is https://auto.ru/cars/used/sale/honda/fit_aria/1105674129-da8f04d7/.
Create a dataframe from the list and concetenate it with the already collected data.
The # of current processing URL is 7050, url is https://auto.ru/cars/used/sale/honda/fit/1105581197-59afcea1/.
The # of current processing URL is 7100, url is https://auto.ru/cars/used/sale/honda/prelude/1105823745-a5b723c1/.
The # of current processing URL is 7150, url is https://auto.ru/cars/used/sale/honda/civic/1105869735-7caf6328/.
The # of current processing URL is 7200, url is https://auto.ru/cars/used/sale/honda/civic_type_r/1105824153-917197f8/.
The # of current processing URL is 7250, url is https://auto.ru/cars/used/sale/honda/accord/1105653101-b443463a/.
The # of current processing URL is 7300, url is https://auto.ru/cars/used/sale/honda/cr_v/1105575021-bbd91f3b/.
The # of current processing URL is 7350, url is https://auto.ru/cars/used/sale/honda/civic/1105259523-6380ca39/.
T

The # of current processing URL is 10450, url is https://auto.ru/cars/used/sale/volvo/xc40/1105681787-d09bd387/.
The # of current processing URL is 10500, url is https://auto.ru/cars/used/sale/volvo/xc90/1105648611-fc85cdf9/.
The # of current processing URL is 10550, url is https://auto.ru/cars/used/sale/volvo/s90/1105516649-58ebbb45/.
The # of current processing URL is 10600, url is https://auto.ru/cars/used/sale/volvo/s60_cross_country/1105043533-60381efd/.
The # of current processing URL is 10650, url is https://auto.ru/cars/used/sale/volvo/s90/1105472916-41468421/.
The # of current processing URL is 10700, url is https://auto.ru/cars/used/sale/volvo/s80/1096223894-e72446ba/.
The # of current processing URL is 10750, url is https://auto.ru/cars/used/sale/volvo/s60/1103614264-fe18170c/.
The # of current processing URL is 10800, url is https://auto.ru/cars/used/sale/volvo/s80/1105791353-f285e4ed/.
The # of current processing URL is 10850, url is https://auto.ru/cars/used/sale/volvo/s9

The # of current processing URL is 14000, url is https://auto.ru/cars/used/sale/bmw/x3/1105702752-7d91ae67/.
Create a dataframe from the list and concetenate it with the already collected data.
The # of current processing URL is 14050, url is https://auto.ru/cars/used/sale/bmw/x5/1105058389-5f05e626/.
The # of current processing URL is 14100, url is https://auto.ru/cars/used/sale/bmw/5er/1105857525-16526d45/.
The # of current processing URL is 14150, url is https://auto.ru/cars/used/sale/bmw/m8/1105853033-716449ff/.
The # of current processing URL is 14200, url is https://auto.ru/cars/used/sale/bmw/6er/1105731194-14f31b26/.
The # of current processing URL is 14250, url is https://auto.ru/cars/used/sale/bmw/1er/1105850285-034985ba/.
The # of current processing URL is 14300, url is https://auto.ru/cars/used/sale/bmw/x2/1105649882-6c3e4c83/.
The # of current processing URL is 14350, url is https://auto.ru/cars/used/sale/bmw/x5/1105844857-90ee0d93/.
The # of current processing URL is 14400

...

The # of current processing URL is 102650, url is https://auto.ru/cars/new/group/volkswagen/tiguan/22680192/22868368/1105847776-e0c0d6b4/.
The # of current processing URL is 102700, url is https://auto.ru/cars/new/group/volkswagen/tiguan/22680193/22688476/1105144112-457b5529/.
The # of current processing URL is 102750, url is https://auto.ru/cars/used/sale/volkswagen/jetta/1105770230-1c4dc5d0/.
The # of current processing URL is 102800, url is https://auto.ru/cars/used/sale/volkswagen/polo/1105833270-a992a3df/.
The # of current processing URL is 102850, url is https://auto.ru/cars/used/sale/volkswagen/touareg/1105669048-f1365fd2/.
The # of current processing URL is 102900, url is https://auto.ru/cars/new/group/volkswagen/tiguan/22680193/22688520/1105607288-a95f7c29/.
The # of current processing URL is 102950, url is https://auto.ru/cars/used/sale/volkswagen/jetta/1105770230-1c4dc5d0/.
The # of current processing URL is 103000, url is https://auto.ru/cars/used/sale/volkswagen/touareg/11

The # of current processing URL is 105850, url is https://auto.ru/cars/used/sale/volkswagen/caravelle/1105664770-49dce837/.
The # of current processing URL is 105900, url is https://auto.ru/cars/used/sale/volkswagen/passat/1104582984-4e5df469/.
The # of current processing URL is 105950, url is https://auto.ru/cars/used/sale/volkswagen/touareg/1105403411-91d877d8/.
The # of current processing URL is 106000, url is https://auto.ru/cars/used/sale/volkswagen/touareg/1105331700-73c2f650/.
Create a dataframe from the list and concetenate it with the already collected data.
The # of current processing URL is 106050, url is https://auto.ru/cars/used/sale/volkswagen/caravelle/1105664770-49dce837/.
The # of current processing URL is 106100, url is https://auto.ru/cars/used/sale/volkswagen/multivan/1105363104-119165f2/.
The # of current processing URL is 106150, url is https://auto.ru/cars/used/sale/volkswagen/polo/1105833270-a992a3df/.
The # of current processing URL is 106200, url is https://au

The # of current processing URL is 109000, url is https://auto.ru/cars/used/sale/mitsubishi/outlander/1105740194-6464152a/.
Create a dataframe from the list and concetenate it with the already collected data.
The # of current processing URL is 109050, url is https://auto.ru/cars/new/group/mitsubishi/outlander/21397560/22971549/1105901751-44ac2aff/.
The # of current processing URL is 109100, url is https://auto.ru/cars/used/sale/mitsubishi/pajero/1104863750-8c285490/.
The # of current processing URL is 109150, url is https://auto.ru/cars/used/sale/mitsubishi/asx/1105768515-0f8880ff/.
The # of current processing URL is 109200, url is https://auto.ru/cars/used/sale/mitsubishi/outlander/1105701786-a5db3e2b/.
The # of current processing URL is 109250, url is https://auto.ru/cars/used/sale/mitsubishi/montero/1105688016-8c102945/.
The # of current processing URL is 109300, url is https://auto.ru/cars/used/sale/mitsubishi/carisma/1105575244-f55c0fbd/.
The # of current processing URL is 109350,

The # of current processing URL is 112100, url is https://auto.ru/cars/used/sale/mitsubishi/pajero_sport/1105683876-42b45b42/.
The # of current processing URL is 112150, url is https://auto.ru/cars/used/sale/mitsubishi/outlander/1105530020-606f9740/.
The # of current processing URL is 112200, url is https://auto.ru/cars/used/sale/mitsubishi/outlander/1105935041-9543c9c5/.
The # of current processing URL is 112250, url is https://auto.ru/cars/used/sale/mitsubishi/montero_sport/1105351181-94bcdc33/.
The # of current processing URL is 112300, url is https://auto.ru/cars/used/sale/mitsubishi/pajero_sport/1105683876-42b45b42/.
The # of current processing URL is 112350, url is https://auto.ru/cars/new/group/mitsubishi/outlander/21397560/22971550/1105732463-03f4328d/.
The # of current processing URL is 112400, url is https://auto.ru/cars/used/sale/mitsubishi/pajero_sport/1105728311-5eb56b18/.
The # of current processing URL is 112450, url is https://auto.ru/cars/used/sale/mitsubishi/lancer/11

The # of current processing URL is 115200, url is https://auto.ru/cars/new/group/mitsubishi/outlander/21397560/22971549/1105866298-f72a28f2/.
The # of current processing URL is 115250, url is https://auto.ru/cars/used/sale/mitsubishi/outlander/1105636253-90bdc4ad/.
The # of current processing URL is 115300, url is https://auto.ru/cars/used/sale/mitsubishi/lancer/1104975498-50c564bb/.
The # of current processing URL is 115350, url is https://auto.ru/cars/new/group/mitsubishi/outlander/21397559/22971553/1105792491-bf6806f9/.
The # of current processing URL is 115400, url is https://auto.ru/cars/used/sale/mitsubishi/outlander/1105636253-90bdc4ad/.
The # of current processing URL is 115450, url is https://auto.ru/cars/used/sale/mitsubishi/lancer/1105647288-95f9b4ed/.
The # of current processing URL is 115500, url is https://auto.ru/cars/used/sale/mitsubishi/outlander/1105530020-606f9740/.
The # of current processing URL is 115550, url is https://auto.ru/cars/used/sale/mitsubishi/pajero_spo

In [40]:
parsed_df_1

,bodyType,brand,car_url,color,complectation_dict,description,engineDisplacement,enginePower,equipment_dict,fuelType,...,vehicleTransmission,vendor,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня,price
0,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,белый,"['cruise-control', 'multi-wheel', 'xenon', 'he...",Автомобиль приобретался у официального дилера ...,1.8 LTR,180 N12,"{'cruise-control': True, 'asr': True, 'tinted-...",бензин,...,роботизированная,NaN,3 или более,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен,999000.0
1,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/rapid/110...,белый,"['cruise-control', 'multi-wheel', 'heated-wash...",Группа компаний Элан Моторс официальный дилер ...,1.6 LTR,110 N12,"{'cruise-control': True, 'glonass': True, 'asr...",бензин,...,автоматическая,NaN,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен,1179000.0
2,лифтбек,SKODA,https://auto.ru/cars/new/group/skoda/rapid/217...,белый,"['cruise-control', 'heated-wash-system', 'airb...",Специальные предложения на автомобили в наличи...,1.6 LTR,90 N12,"{'cruise-control': True, 'glonass': True, 'esp...",бензин,...,механическая,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1464100.0
3,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,синий,"['cruise-control', 'multi-wheel', 'heated-wash...",Купим Ваш автомобиль ДОРОГО Гарантированная с...,1.4 LTR,150 N12,"{'cruise-control': True, 'esp': True, 'usb': T...",бензин,...,роботизированная,NaN,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен,1420000.0
4,внедорожник 5 дв.,SKODA,https://auto.ru/cars/new/group/skoda/karoq/217...,серый,"['cruise-control', 'multi-wheel', 'heated-wash...",ЛОТ 01267595 Скидка на автомобиль при покупке ...,1.4 LTR,150 N12,"{'cruise-control': True, 'asr': True, 'esp': T...",бензин,...,автоматическая,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2653190.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130196,внедорожник 5 дв.,MITSUBISHI,https://auto.ru/cars/new/group/mitsubishi/outl...,белый,"['cruise-control', 'multi-wheel', 'airbag-pass...",ФУНКЦИОНАЛЬНОЕ ОБОРУДОВАНИЕ Центральный замок...,2.0 LTR,146 N12,"{'cruise-control': True, 'asr': True, 'esp': T...",бензин,...,вариатор,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2039175.0
130197,внедорожник 5 дв.,MITSUBISHI,https://auto.ru/cars/used/sale/mitsubishi/outl...,синий,"['xenon', 'airbag-passenger', 'lock', 'electro...",Автомобиль куплен в 2012 году у официального д...,2.4 LTR,170 N12,"{'asr': True, 'tinted-glass': True, 'esp': Tru...",бензин,...,вариатор,NaN,1 владелец,NaN,Оригинал,полный,Левый,Не требует ремонта,Растаможен,978000.0
130198,седан,MITSUBISHI,https://auto.ru/cars/used/sale/mitsubishi/lanc...,серебристый,"['multi-wheel', 'airbag-passenger', 'lock', 'e...",ТРЕЙД ИН Выгода при обмене на Ваш авто от 30 ...,1.6 LTR,117 N12,"{'airbag-driver': True, 'isofix': True, 'aux':...",бензин,...,механическая,NaN,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен,875500.0
130199,седан,MITSUBISHI,https://auto.ru/cars/used/sale/mitsubishi/lanc...,синий,NaN,Яркий надежный Японский седан MITSUBISHI LANCE...,1.6 LTR,98 N12,"{'audiosystem-cd': True, 'tinted-glass': True,...",бензин,...,автоматическая,NaN,3 или более,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен,NaN


In [41]:
parsed_df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130201 entries, 0 to 130200
Data columns (total 33 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   bodyType              129986 non-null  object 
 1   brand                 129988 non-null  object 
 2   car_url               129969 non-null  object 
 3   color                 129988 non-null  object 
 4   complectation_dict    105267 non-null  object 
 5   description           129988 non-null  object 
 6   engineDisplacement    129988 non-null  object 
 7   enginePower           129988 non-null  object 
 8   equipment_dict        129969 non-null  object 
 9   fuelType              129986 non-null  object 
 10  image                 130065 non-null  object 
 11  mileage               103034 non-null  float64
 12  modelDate             129986 non-null  float64
 13  model_info            0 non-null       float64
 14  model_name            103034 non-null  object 
 15  